# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
import nltk

import sys
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV


from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, recall_score, precision_score
from wordcloud import WordCloud, STOPWORDS 

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database

engine = create_engine('sqlite:///InsertDatabaseName.db')

df = pd.read_sql("SELECT * FROM InsertTableName", engine)


In [3]:
X = df['message']

# categories are all columns after the 4th column
Y = df.iloc[:,4:]

### 2. Tokenization function to process  text data

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build Machine learning pipeline
This machine learning pipeline takes in the `message` column as input, and outputs the classification results on the other 36 categories in the dataset using the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) classifier for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

In [7]:
# train pipeline
pipeline.fit(X_train, Y_train)
model = pipeline

### 5. Test  model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# overall accuracy

Y_pred = model.predict(X_test)

# overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))


# If some labels are not predicted at least once, Y_pred will have different 
# columns than Y_test, which will cause an error in the classification_report()
# So make sure Y_pred has the same labels as Y_test.

Y_pred = pd.DataFrame(Y_pred);
Y_pred.columns = Y_test.columns;
Y_pred.index = Y_test.index;

print(classification_report(Y_test, Y_pred, target_names=Y_pred.columns))

Accuracy 92.94% 

                        precision    recall  f1-score   support

               related       0.82      0.84      0.83      3998
               request       0.57      0.55      0.56       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.64      0.61      0.62      2164
          medical_help       0.33      0.31      0.32       435
      medical_products       0.39      0.33      0.35       279
     search_and_rescue       0.23      0.21      0.22       136
              security       0.07      0.06      0.07        96
              military       0.38      0.37      0.38       158
                 water       0.65      0.64      0.64       335
                  food       0.75      0.73      0.74       584
               shelter       0.63      0.61      0.62       468
              clothing       0.47      0.51      0.49        70
                 money       0.35      0.35      0.35       112
        missing_peopl

In [9]:
# test msg

msg = ['I need help, my house is destroyed']
test_output = pipeline.predict(msg)
print(Y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'shelter' 'infrastructure_related'
 'buildings' 'other_infrastructure' 'direct_report']


### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])


# check pipeline parameters
# pipeline.get_params()

parameters = {
        'clf__estimator__n_estimators': [10, 20],
        'clf__estimator__learning_rate': [0.5, 1]

    } 

cv = GridSearchCV(estimator=pipeline, n_jobs = -1, param_grid=parameters)

In [22]:
cv.fit(X_train, Y_train)
model = cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  


In [25]:
# overall accuracy

Y_pred = model.predict(X_test)

# overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))


# If some labels are not predicted at least once, Y_pred will have different 
# columns than Y_test, which will cause an error in the classification_report()
# So make sure Y_pred has the same labels as Y_test.

Y_pred = pd.DataFrame(Y_pred);
Y_pred.columns = Y_test.columns;
Y_pred.index = Y_test.index;

print(classification_report(Y_test, Y_pred, target_names=Y_pred.columns))

print("\nBest Parameters:", model.best_params_)

Accuracy 94.55% 

                        precision    recall  f1-score   support

               related       0.81      0.95      0.88      3998
               request       0.78      0.48      0.60       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.74      0.55      0.63      2164
          medical_help       0.61      0.25      0.35       435
      medical_products       0.70      0.27      0.39       279
     search_and_rescue       0.69      0.15      0.24       136
              security       0.14      0.02      0.04        96
              military       0.53      0.22      0.31       158
                 water       0.74      0.58      0.65       335
                  food       0.82      0.66      0.73       584
               shelter       0.77      0.46      0.57       468
              clothing       0.76      0.46      0.57        70
                 money       0.54      0.29      0.37       112
        missing_peopl

### 8. Improve  model further:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
pipeline = Pipeline([

        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LinearSVC()))
])

parameters = {
        'clf__estimator__loss': ('hinge', 'squared_hinge'),
        'clf__estimator__C': (0.5, 1.0)
    } 


cv = GridSearchCV(estimator=pipeline, n_jobs = -1, param_grid=parameters)

In [15]:
# train pipeline
cv.fit(X_train, Y_train)
model = cv

In [20]:
# overall accuracy

Y_pred = model.predict(X_test)

# overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))


# If some labels are not predicted at least once, Y_pred will have different 
# columns than Y_test, which will cause an error in the classification_report()
# So make sure Y_pred has the same labels as Y_test.

Y_pred = pd.DataFrame(Y_pred);
Y_pred.columns = Y_test.columns;
Y_pred.index = Y_test.index;

print(classification_report(Y_test, Y_pred, target_names=Y_pred.columns))

print("\nBest Parameters:", model.best_params_)

Accuracy 95.13% 

                        precision    recall  f1-score   support

               related       0.86      0.93      0.90      3998
               request       0.80      0.59      0.68       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.76      0.70      0.73      2164
          medical_help       0.65      0.23      0.34       435
      medical_products       0.72      0.25      0.37       279
     search_and_rescue       0.67      0.16      0.26       136
              security       0.00      0.00      0.00        96
              military       0.58      0.21      0.31       158
                 water       0.76      0.65      0.70       335
                  food       0.82      0.77      0.79       584
               shelter       0.81      0.56      0.66       468
              clothing       0.70      0.47      0.56        70
                 money       0.68      0.17      0.27       112
        missing_peopl

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [19]:
m = pickle.dumps('classifier.pkl')